In [1]:
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra

import src.models.yolov5

In [9]:
with initialize(config_path="../configs"):
    cfg = compose(config_name='train', overrides=['module=yolov5', 'module.backbone=small_p6', 'module.model.model.device.device=cpu'])
    print(OmegaConf.to_yaml(cfg))

train: true
test: true
seed: 12
name: null
trainer:
  _target_: pytorch_lightning.Trainer
  gpus: 1
  accumulate_grad_batches: 4
  min_epochs: 60
module:
  model_type: yolov5
  backbone: small_p6
  img_size:
  - 1024
  - 896
  pretrained: null
  model:
    _target_: src.modules.yolov5.YoloV5Module
    model:
      _target_: src.models.yolov5.model
      device:
        _target_: torch.device
        device: cpu
      backbone:
        _target_: src.models.yolov5.backbones.${module.backbone}
        pretrained: 1
      num_classes: 2
      img_size: 1024
    learning_rate: 0.0001
    weight_decay: 1.0e-06
datamodule:
  _target_: src.datamodules.dental.DentalCaries
  data_root: /datagrid/personal/kuntluka/dental_rtg
  model_type: ${module.model_type}
  ann_file: caries6.json
  batch_size: 1
  num_workers: 8
  limit_train_samples: null
  train_val_test_split:
  - 0.7
  - 0.15
  - 0.15
  seed: ${seed}
logger:
  wandb:
    _target_: pytorch_lightning.loggers.wandb.WandbLogger
    project: M

In [10]:
model = hydra.utils.instantiate(cfg.module.model)

In [18]:
from transforms.albumentations_adapter import Adapter
from transforms.albumentations_utils import resize_and_pad
import albumentations as A
tfms = resize_and_pad((1024,896))

src.models.yolov5.infer_dl(ds, Adapter(tfms))

NameError: name 'src' is not defined

In [9]:

img = Image.open('../samples/dataset/images/1.png')
ds = Dataset.from_images(images=[img])

In [10]:
ds[0]

BaseRecord

common: 
	- Image size ImgSize(width=1068, height=847)
	- Img: 1068x847x4 <np.ndarray> Image
	- Record ID: 0
detection: 
	- Class Map: None

In [15]:
# import src.modules.yolov5
with hydra.initialize(config_path='../configs'):
    cfg = hydra.compose(config_name='train', overrides=['module=yolov5', 'datamodule.batch_size=1'])
model = hydra.utils.instantiate(cfg.module.model)
# model._get_lightning_module_state_dict(),


MissingConfigException: In 'hydra/config': Could not find 'hydra/job_logging/colorlog'

Available options in 'hydra/job_logging':
	default
	disabled
	none
	stdout
Config search path:
	provider=hydra, path=pkg://hydra.conf
	provider=main, path=file:///home/kuntik/dev/MT/configs
	provider=schema, path=structured://